In [28]:
import pandas as pd
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

#collecting the dataset from drive and split into tarining and validation set

In [24]:
pip install split-folders

In [16]:
url='https://drive.google.com/file/d/12cy6nEEDAfL6isqnTlJIRy8NZLPtIMpE/view?usp=sharing'
data=tf.keras.utils.get_file('archive.zip',extract=True,
                            origin=url)
path=os.path.join(os.path.dirname(data),'plantvillage dataset')


In [ ]:
!unzip '/content/drive/MyDrive/Capstone_project/archive.zip'

In [ ]:
import splitfolders  # or import split_folders

# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio('/content/plantvillage dataset/segmented', output="output", seed=1337, ratio=(.8, .2), group_prefix=None) # default values

In [29]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('./output/train', # this is where you specify the path to the main data folder
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 43430 images belonging to 38 classes.


In [30]:
val_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

val_generator=train_datagen.flow_from_directory('./output/val', # this is where you specify the path to the main data folder
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 10876 images belonging to 38 classes.
